In [ ]:
from datetime import datetime
start_time = datetime.now() # initialize timer
import matplotlib.pyplot as pl
import numpy as np
import scipy.optimize as op
import os
import astropy.units as u
import sys
from scipy.stats import chisquare
from datetime import datetime
from astropy.coordinates import SkyCoord
import pandas as pd
from scipy import stats
from scipy.optimize import curve_fit
import emcee

In [ ]:
class color:
   PURPLE = '\033[1;35;48m'
   CYAN = '\033[1;36;48m'
   BOLD = '\033[1;37;48m'
   BLUE = '\033[1;34;48m'
   GREEN = '\033[1;32;48m'
   YELLOW = '\033[1;33;48m'
   RED = '\033[1;31;48m'
   BLACK = '\033[1;30;48m'
   UNDERLINE = '\033[4;37;48m'
   END = '\033[1;37;0m'

In [ ]:
local_path='/Users/adrianhernandez/JPL_2021/'
sys.path.append(local_path + 'MulensModel_master/source')
import MulensModel as mm
from MulensModel.utils import Utils
import mcmcFit as mcfit

In [ ]:
##import data 

from glob import glob
im_dir = '/Users/adrianhernandez/JPL_2021/ukirt_files/'
ukirt_file = glob(f"{im_dir}/*C28997*.txt")
print(ukirt_file[0]) #change indicies to update the file you are interested 
# filename = ukirt_file[2][44:76]
# ukirt_data_H = np.loadtxt(ukirt_file[0], usecols=range(3))
# ukirt_data_K = np.loadtxt(ukirt_file[2], usecols=range(3))
# H_band = mm.MulensData(file_name=ukirt_file[0])
# K_band = mm.MulensData(file_name=ukirt_file[2])

# print(filename)

In [ ]:
# Plot the data
pl.figure(figsize=(10,10))
pl.errorbar(H_band.time-2450000, H_band.mag, yerr=H_band.err_mag, fmt='.r')
pl.errorbar(K_band.time-2450000, K_band.mag, yerr=K_band.err_mag, fmt='.b')
pl.gca().invert_yaxis()
pl.show()

# ***Define plot limits for a zoom (of the planetary perturbation)***
(t_min, t_max) = (8575,8800)

# Plot a zoom of the data
pl.figure(figsize=(10,10))
pl.errorbar(H_band.time-2450000, H_band.mag, yerr=H_band.err_mag, fmt='.r')
pl.errorbar(K_band.time-2450000, K_band.mag, yerr=K_band.err_mag, fmt='.b')
pl.xlim(t_min, t_max)
pl.xlabel('t - 2450000')
# pl.axvline(x = 8300)
# pl.axvline(x = 8288)
# pl.axvline(x = 8286.99471) 
pl.gca().invert_yaxis()
pl.show()
pl.figure(figsize=(10,10))
pl.errorbar(H_band.time-2450000, H_band.mag, yerr=H_band.err_mag, fmt='.r')
pl.errorbar(K_band.time-2450000, K_band.mag, yerr=K_band.err_mag, fmt='.b')
pl.xlim(t_min, t_max)
pl.xlabel('t - 2450000')
pl.xlim(8649,8680)
pl.gca().invert_yaxis()
pl.show()

In [ ]:
# ***Set time range of planetary perturbation (including 2450000).***
(t_planet_start, t_planet_stop) = (2458649,2458690)

# ***Set time range of planetary perturbation (including 2450000).***

# *Set the magnification methods for the planet model*
# VBBL method will be used between t_planet_start and t_planet_stop, 
# and point_source_point_lens will be used everywhere else.
magnification_methods = [  t_planet_start, 'VBBL', t_planet_stop, 'point_source_point_lens']

In [ ]:
t0,u0,tE = (8654.988477399034, 0.020000 , 120.0000)


In [ ]:

point_lens_model = mm.Model({'t_0': t0+2450000, 'u_0':u0, 't_E': tE})
point_lens_event = mm.Event(datasets=[H_band,K_band], model=point_lens_model)
pl.figure(figsize=(10,10))
point_lens_event.plot_model(
    t_range=[2458575,2458800], subtract_2450000=True, color='black')

point_lens_event.plot_data(subtract_2450000=True, color_list=['cyan', 'red'],label_list= ['H_band','K_band'],  s=5)
pl.xlim(8575,8800)
pl.legend(loc='best')
print('PSPL Fitted Parameters')
print(point_lens_model)

In [ ]:
print(point_lens_event.model.parameters.t_0)
print(point_lens_event.model.parameters.u_0)
print(point_lens_event.model.parameters.t_E)

In [ ]:
# Estimate s (projected separation) of the planet, alpha (angle of source trajectory)

# Approximate time of the planetary perturbation
t_planet = (t_planet_stop + t_planet_start) / 2. # this important

# Position of the source at the time of the planetary perturbation
tau_planet = ((t_planet - point_lens_event.model.parameters.t_0) /
              point_lens_event.model.parameters.t_E)

u_planet = np.sqrt(point_lens_event.model.parameters.u_0**2 + tau_planet**2)

# Position of the lens images at the time of the planetary perturbation
# --> Estimate of the planet location
s_minus = 0.5 * (np.sqrt(u_planet**2 + 4.) - u_planet)
s_plus = 0.5 * (np.sqrt(u_planet**2 + 4.) + u_planet)

# Angle between the source trajectory and the binary axis

alpha_planet = np.rad2deg(-np.arctan2(point_lens_event.model.parameters.u_0, tau_planet))#-180 deg?

print(t_planet)
print(tau_planet)
print(u_planet)
print(s_plus,s_minus)
print(alpha_planet)

In [ ]:
# Test model with estimated planetary parameters
# Check the estimated model
# Note that there are two possibilities for s: s_plus and s_minus. 
# Only s_plus is tested here, but both are considered in the grid search below.

# Define the model
test_model = mm.Model({
    't_0': 2458654.98848, 
    'u_0': 0.05,
    't_E': 62,
    'rho': 0.025,
    's':  1.06086, #effects the shape of the causitc 
    'q': 0.018,
    'alpha': -240.0})
test_model.set_magnification_methods(magnification_methods)
test_event = mm.Event(datasets=[H_band,K_band], model=test_model)

print(test_event.model)

# Plot the model light curve
pl.figure(figsize=(10,10))
test_event.plot_data(subtract_2450000=True, color_list=['cyan', 'red'],label_list= ['H_band','K_band'],  s=5)
test_event.plot_model(t_range=[2457900,2458800], subtract_2450000=True, color='black')
#pl.xlim(8600,8800)
pl.title(filename + ' initial fit')
#pl.savefig(filename + '_initial_fit', dpi=100)
pl.show()

pl.figure(figsize=(10,10))
test_event.plot_data(subtract_2450000=True, color_list=['cyan', 'red'],label_list= ['H_band','K_band'],  s=5)
test_event.plot_model(t_range=[2458575,2458800], subtract_2450000=True, color='black')
pl.xlim(8600,8725)
pl.title(filename + ' initial fit with planetary zoom')
#pl.savefig(filename + '_planetary zoom', dpi=100)
pl.show()

#Plot the trajectory of the source relative to the caustics
test_event.model.plot_trajectory(color='black', caustics=True)
pl.xlim(-.1,.25)
pl.ylim(-.2,.2)
pl.title(filename + ' caustic')
#pl.savefig(filename + '_caustic', dpi=100)
pl.show()

In [ ]:
## np.set_printoptions(threshold=sys.maxsize)

#Define fitting functions

def chi2_fun(theta, event, parameters_to_fit):
    """                                                                         
    Chi2 function. Changes values of the parameters and recalculates chi2.
    
    event = a MulensModel.Event
    parameters_to_fit = list of names of parameters to be changed
    theta = values of the corresponding parameters
    """
    # key = the name of the MulensModel parameter
    for (index, key) in enumerate(parameters_to_fit):
        if (key == 't_E' or key =='rho') and theta[index] < 0.:
            return np.inf 
        setattr(event.model.parameters, key, theta[index])
    return event.get_chi2(fit_blending = True)

def fit_model(event, parameters_to_fit):
    """
    Fit an "event" with "parameters_to_fit" as free parameters.
    
    event = a MulensModel event
    parameters_to_fit = list of parameters to fit
    """
    # Take the initial starting point from the event.
    x0 = []
    for key in parameters_to_fit:
        value = getattr(event.model.parameters, key)
        if isinstance(value, u.Quantity):
            x0.append(value.value)
        else:
            x0.append(value)

    # *Execute fit using a 'Nelder-Mead' algorithm*
    result = op.minimize(
        chi2_fun, x0=x0, args=(event, parameters_to_fit),
        method='Nelder-Mead')

    return result

In [ ]:
# Using the Point Lens fit as input, search for a planetary solution
#
# Grid parameters: s (log), q (log)
# Fit parameters: rho, alpha
# PSPL parameters: t_0, u_0, t_E
#

# *Define the grid*
delta_log_s = 0.01
delta_log_q = 0.05
grid_log_s = np.hstack(
    (np.arange(
        np.log10(s_minus) - .02, np.log10(s_minus) + 0.02, delta_log_s),
    np.arange(
        np.log10(s_plus) - 0.02, np.log10(s_plus) + 0.02, delta_log_s)))
grid_log_q = np.arange(-2.9, -2.1, delta_log_q)

print(grid_log_s)
print(grid_log_q)